In [1]:
import pandas as pd
from konlpy.tag import Okt

In [13]:
from sklearn.model_selection import train_test_split

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [3]:
okt = Okt()

In [2]:
positive_dict = pd.read_csv('positive_score_dict.csv')
positive_dict = positive_dict.drop_duplicates(subset='단어')
positive_dict.set_index('단어', inplace=True)

result2 = pd.read_csv('community_token_final.csv', low_memory=False)

In [4]:
# 각 열에 대해 okt.morphs(stem=True) 적용 및 문자열 변환

result2['형용사_어간'] = result2['내용_형용사_str'].apply(
lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

result2['부사_어간'] = result2['내용_부사_str'].apply(
lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

result2['동사_어간'] = result2['내용_동사_str'].apply(
lambda x: ', '.join(okt.morphs(x, stem=True)) if isinstance(x, str) and x != 'NaN' else ''
)

In [7]:
result2.head(1)

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str,형용사_어간,부사_어간,동사_어간,긍정점수
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다","둥글다, ,, 있다, ,, 동일하다, ,, 있다",,"되어다, ,, 오다, ,, 보이다",-1


In [ ]:
# 행별 총 긍정 점수를 계산할 파생열 추가
row_scores = [0] * len(result2)  # 초기화

for idx in range(len(result2)):  # 데이터프레임 행 반복
    row_score = 0  # 행별 총 점수 초기화

    for column in ['내용_명사_str', '형용사_어간', '부사_어간', '동사_어간']:
        word_list = result2.loc[idx, column]  # 각 열의 데이터 가져오기
        print(f"\n행 {idx}, 열 {column} 처리 중: {word_list}")
        
        if pd.isna(word_list) or word_list.strip() == "":  # Null 또는 빈 문자열 처리
            print(f"행 {idx}, 열 {column}은 결측값 또는 빈 문자열입니다. 건너뜁니다.")
            continue
        
        words = word_list.split(', ')  # 쉼표와 공백으로 구분
        for word in words:
            word = word.strip()  # 공백 제거
            if word in positive_dict.index:
                score = positive_dict.loc[word, '점수']
                row_score += score  # 점수 합산
                print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
            else:
                print(f"  단어: {word}는 사전에 없음")

    row_scores[idx] = row_score  # 행별 총 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 점수 변환: 음수는 -1, 0은 0, 양수는 1
try:
    result2['긍정점수'] = [1 if score > 0 else 0 if score < 0 else -1 for score in row_scores]
except Exception as e:
    print(f"점수 변환 중 에러 발생: {e}")
    print(f"row_scores 내용: {row_scores}")
    print(f"tmp 길이: {len(result2)}, row_scores 길이: {len(row_scores)}")

# 최종 점수 출력
print("\n행별 긍정점수:")
print(result2[['긍정점수']])

In [11]:
result2['긍정점수'].value_counts()

긍정점수
-1    52095
 0    15583
 1    13394
Name: count, dtype: int64

In [12]:
# 언더 샘플링

sample_data_positive = result2[result2['긍정점수'] == 1].sample(13394)
sample_data_negative = result2[result2['긍정점수'] == 0].sample(13394)

total_data = pd.concat([sample_data_positive,sample_data_negative])

In [14]:
# 학습 데이터와, 테스트 데이터를 구분

train_data, test_data = train_test_split(total_data, test_size=0.25, random_state=2025)

In [17]:
# TF-IDF 변환

vectorizer = TfidfVectorizer()
# 학습 데이터 벡터화

X_train = vectorizer.fit_transform(train_data['내용'])
y_train = train_data['긍정점수'].values
# 테스트 데이터 벡터화

X_test = vectorizer.transform(test_data['내용'])
y_test = test_data['긍정점수'].values

In [19]:
# 모델 구축

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))  # 추가된 레이어
model.add(Dropout(0.5))  # 드롭아웃 추가
model.add(Dense(1, activation='sigmoid'))  # 이진 분류

d:\kdt\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 조기 종료 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])  # 20%를 검증 데이터로 사용

Epoch 1/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.6352 - loss: 0.6239 - val_accuracy: 0.7803 - val_loss: 0.4274
Epoch 2/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.9472 - loss: 0.1616 - val_accuracy: 0.7890 - val_loss: 0.4710
Epoch 3/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.9966 - loss: 0.0189 - val_accuracy: 0.7952 - val_loss: 0.5907
Epoch 4/20
503/503 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.9997 - loss: 0.0032 - val_accuracy: 0.7962 - val_loss: 0.6460


In [21]:
# 모델 평가

y_pred = model.predict(X_test)
y_pred_classes = [1 if p > 0.5 else 0 for p in y_pred]

210/210 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


In [22]:
# 성능 평가
print(classification_report(y_test, y_pred_classes))

              precision    recall  f1-score   support

           0       0.79      0.79      0.79      3435
           1       0.78      0.78      0.78      3262

    accuracy                           0.79      6697
   macro avg       0.79      0.79      0.79      6697
weighted avg       0.79      0.79      0.79      6697

